## 연합인포맥스 '금리'관련 뉴스 수집

In [1]:
import requests
import sqlite3
from bs4 import BeautifulSoup
import datetime 
import wget
import html2text

In [2]:
pgno=1
mainurl = f'http://news.einfomax.co.kr/news/articleList.html?page={pgno}&total=122257&box_idxno=&sc_area=A&view_type=sm&sc_word=금리'


In [3]:
dbname='einfomax_news.db'
def sqexe(s,data):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        result=cur.execute(s,data).fetchall()
        conn.commit()
        cur.close()
    return result

def mknewsTable(title,wday,content,comp='인포맥스',cate='경제'):
    sql="select title from einfomax where title=?"
    # title이 같은 기사는 없을 것이라는 가정
    data=[title]
    res=sqexe(sql,data)
    
    if (res):
        print('uduplicate')
    else:            
        sql="""insert into einfomax (title,wday,content,comp,cate) 
        values(?,?,?,?,?)"""
        data=[title,wday,content,comp,cate]
        res=sqexe(sql,data)


In [8]:
import re

url=mainurl
preurl='http://news.einfomax.co.kr'
#pdf_folder='pdfs/'
def IrGather(fr=1,to=10):
    for i in range(fr,to):
        ur=f'http://news.einfomax.co.kr/news/articleList.html?page={i}&total=122257&box_idxno=&sc_area=A&view_type=sm&sc_word=금리'
        urlGet(ur)
        
def urlGetSub(url):
    subpage=requests.get(url)
    soup2 = BeautifulSoup(subpage.content, 'html.parser')
    title = soup2.select('.article-head-title')[0].get_text().strip()
    title = re.sub('\[.*\]|<.*>', '', title)
    
    wday = soup2.select('.info-text li')[1].get_text().replace('\t','')[6:14].replace('.','')
    
    con = soup2.select('#article-view-content-div')
    content = ' '.join(str(v.get_text()) for v in con)
    
    content = re.sub('[가-힣]{2,8} 기자', '', content)
    content = re.sub('\t', '', content).strip()
    content = re.sub('\[.*\]|<.*>', '', content)
    content = re.sub('\(.*\).*= ', '', content)
    content = re.sub('\(.*연합인포맥스.*\)', '', content)
    email_pattern = '[a-zA-Z0-9]*@[a-zA-Z]*\.[a-zA-Z]*(\.[a-zA-Z]*)?'
    content = re.sub(email_pattern, '', content)
    content = re.sub('http.*', '', content)
    content = re.sub('▲', '', content)
    content = re.sub('\(끝\).*', '', content)
    content = re.sub('저작권자 ©.*', '', content)
    content = re.sub('본 기사는 인포맥스.*', '', content).strip()
    
    print(title)
    print(wday)
    
    return {"t":title, "d":wday, "c":content}
    
    
def urlGet(url):
    headers={'Content-Tyze': 'application/json; charset=utf-8',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/76.0.3809.100'}
    page=requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content,'html.parser')
    rows=soup.select('.article-list .list-block')
    for r in rows:
        title=r.select('.list-titles')[0].get_text().strip().replace('.','')
        link=r.select('a')[0]['href'].strip()
        if "일정" not in title and "<표>" not in title and "[표]" not in title:
            print(preurl+link)
            res = urlGetSub(preurl+link)
            mknewsTable(res['t'], res['d'], res['c'])
        print('*'*50)
        
        
# 1년치 523p 까지 / 2년치 1035p
IrGather(1001,1035)

http://news.einfomax.co.kr/news/articleView.html?idxno=3412752
모건스탠리 "달러화, 트럼프 세제개편안 수혜"
170919
**************************************************
http://news.einfomax.co.kr/news/articleView.html?idxno=3412748
BOE 총재 "브렉시트, 영국 물가 상승 압력 높여"(상보)
170919
**************************************************
http://news.einfomax.co.kr/news/articleView.html?idxno=3412747
JP모건 "독일·한국 투자자, 영국 부동산 시장에 몰릴 것"
170919
**************************************************
http://news.einfomax.co.kr/news/articleView.html?idxno=3412744
골드만 삭스 "연준 자산매각 최대 수혜 주는 은행"
170919
**************************************************
http://news.einfomax.co.kr/news/articleView.html?idxno=3412743
달러화, 엔화에 상승 지속…유로화에 하락
170919
**************************************************
http://news.einfomax.co.kr/news/articleView.html?idxno=3412742
BOE 총재 "몇달 내 금리 인상 필요할수도…브렉시트 투자·이민·생산성에 부담"
170919
**************************************************
http://news.einfomax.co.kr/news/articleView.html?idxno=3412740
미 국채가, FOMC 앞두고 하락 지

In [16]:
# SQLite DB에서 pandas DataFrame으로 읽어오기
import pandas as pd
import sqlite3

con = sqlite3.connect(dbname)
df = pd.read_sql("SELECT * FROM einfomax", con, index_col=None)
df.head()

,idx,cate,title,content,wday,comp
0,1,경제,"""서민형 안심전환대출, 우대금리 적용시 최저 연 1.2%대""",신혼·다자녀 가구 온라인 신청시 연 1.25%까지\n\n금융위원회가 오는 9월 최저...,190825,인포맥스
1,2,경제,1%대 서민형 안심전환대출 20조 공급…내달 16일 출시,"소득 8천500만원 이하 1주택자, 연 1.85~2.2% 금리로 '갈아타기'만 가능...",190825,인포맥스
2,3,경제,무역전쟁 격랑…경기 침체 공포 지속,이번 주(26~30일) 뉴욕증시는 미·중 무역전쟁 격화 여파로 불안정한 흐름을 이어...,190825,인포맥스
3,4,경제,미·중 무역전쟁 격화…주가 급락·달러↓국채↑,23일(미국시간) 뉴욕증시에서 주요 지수는 미국과 중국의 무역전쟁 격화와 제롬 파월...,190824,인포맥스
4,5,경제,딜링룸 24시…챙겨봐야할 국제경제뉴스,"-트럼프 ""중국 필요 없다…기업에 中 대체처 찾도록 지시""(상보)\n도널드 트럼프 ...",190824,인포맥스
